In [2]:
import jieba
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import matplotlib
import pymysql
from sklearn.feature_extraction.text import CountVectorizer
import re

from configs import USERNAME, PASSWORD, DBNAME

matplotlib.rcParams['font.sans-serif'] = ["simHei"]
matplotlib.rcParams['font.family'] = 'sans-serif'

In [3]:
sql = "SELECT * FROM jobs"
connect = pymysql.connect(host="127.0.0.1", user=USERNAME, password=PASSWORD, db=DBNAME)
job_data = pd.read_sql(sql, connect)
connect.close()

data = job_data[["company_id", "position_name", "salary"]]
del job_data


In [4]:
def deal_word(word):
    if word is not None:
        generator_ = jieba.cut(word)
        return " ".join(generator_)

def del_stopwords(words, stopwords):
    if words is not None:
        output = ''
        for word in words:
            if word not in stopwords:
                output += word
        return output

stopwords = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '’', '!', '"', '#', '$', '¥', '%', '&', '\\', "'", '(',
             ')', '（', '）', '*', '+', ',', '-', '.', '/', ':', '：', ';', '<', '=', '>', '?', '@', '，', '。', '?', '★',
             '、', '…', '【', '】', '〖', '〗', '《', '》', '？', '“', '”', '‘', '’', '！', '[', '\\', '\\', ']', '^', '_',
             '`', '{', '|', '}', '~', '', '', "工程师"]

data["describe_cut"] = data["position_name"].apply(deal_word).apply(del_stopwords, stopwords=stopwords)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\123\AppData\Local\Temp\jieba.cache
Loading model cost 0.724 seconds.
Prefix dict has been built successfully.


In [5]:
data

,company_id,position_name,salary,describe_cut
0,53,中级/高级DBA,16k-30k,中级 高级 DBA
1,53,PHP开发工程师,15k-25k,PHP 开发 工程师
2,53,IT工程师,5k-8k,IT 工程师
3,53,IT技术支持,5k-7k,IT 技术支持
4,53,python开发工程师,25k-30k,python 开发 工程师
...,...,...,...,...
130439,122184318,软件高级开发工程师,8k-15k,软件 高级 开发 工程师
130440,122184318,软件高级开发工程师,8k-15k,软件 高级 开发 工程师
130441,122184318,UI高级设计,11k-12k,UI 高级 设计
130442,122185826,vivo手机销售顾问,4k-8k,vivo 手机 销售 顾问


In [6]:
describe_s = data.groupby("company_id")["describe_cut"].apply(" ".join)


In [7]:
count_vec = CountVectorizer(min_df=5, max_df=5000, max_features=100,stop_words=stopwords)
sparse_result_count = count_vec.fit_transform(describe_s)
# 输出稀疏矩阵
print(sparse_result_count)
# 输出关键词
print('\nvocabulary list:\n\n',count_vec.get_feature_names())
# 输出关键词编号
print('\nvocabulary dic:\n\n',count_vec.vocabulary_)

  (0, 15)	2
  (0, 99)	2
  (0, 7)	4
  (0, 47)	6
  (0, 25)	2
  (0, 17)	2
  (0, 54)	2
  (0, 23)	2
  (0, 32)	2
  (0, 45)	2
  (0, 18)	2
  (1, 47)	8
  (1, 25)	2
  (1, 17)	4
  (1, 32)	2
  (1, 18)	6
  (1, 43)	6
  (1, 11)	6
  (1, 26)	4
  (1, 9)	2
  (1, 5)	10
  (1, 29)	8
  (1, 77)	12
  (1, 93)	6
  (1, 52)	2
  :	:
  (6915, 11)	1
  (6915, 78)	1
  (6915, 19)	1
  (6916, 11)	2
  (6916, 52)	1
  (6916, 95)	2
  (6916, 20)	1
  (6916, 19)	1
  (6918, 16)	2
  (6919, 28)	1
  (6920, 47)	1
  (6920, 1)	1
  (6921, 17)	1
  (6921, 43)	1
  (6921, 77)	2
  (6922, 98)	1
  (6922, 88)	1
  (6923, 47)	2
  (6923, 1)	1
  (6924, 99)	3
  (6924, 47)	2
  (6924, 8)	1
  (6924, 91)	2
  (6925, 98)	1
  (6925, 95)	1

vocabulary list:

 ['ai', 'android', 'bd', 'hrbp', 'ios', 'java', 'mj', 'php', 'ui', 'web', '上海', '专员', '专家', '业务', '中心', '中级', '主播', '主管', '产品', '人事', '代表', '优化', '内容', '分析师', '前端', '前端开发', '助理', '北京', '司机', '后台', '品牌', '售前', '商业', '商业化', '商务', '培训', '增长', '媒体', '安全', '实习生', '客户', '客户端', '客户经理', '客服', '市场', '平台', '广告', 

In [8]:
sparse_result_count.toarray()

array([[  0,   0,   0, ...,   0,   0,   2],
       [  0,   0,   0, ...,   0,   0,   0],
       [ 94, 200,  12, ...,  28, 244, 982],
       ...,
       [  0,   0,   0, ...,   0,   0,   3],
       [  0,   0,   0, ...,   0,   1,   0],
       [  0,   0,   0, ...,   0,   0,   0]], dtype=int64)

In [9]:
print(sparse_result_count.shape)


(6927, 100)


In [10]:
dict_word = dict()
n = 0
for word in count_vec.get_feature_names():

    dict_word[word] = dict_word.get(word, [])
    for i in range(len(data["describe_cut"])):
        if word in data["describe_cut"][i]:
            dict_word[word].append(i)
    n += 1
    if n % 10 == 0:
        print("ok + 1, current is {}".format(n) )

ok + 1, current is 10
ok + 1, current is 20
ok + 1, current is 30
ok + 1, current is 40
ok + 1, current is 50
ok + 1, current is 60
ok + 1, current is 70
ok + 1, current is 80
ok + 1, current is 90
ok + 1, current is 100


In [11]:
def deal(x: str, pattern: str, i: int):
  result = re.search(pattern, str(x))
  return result.groups()[i] if result is not None else 0

data['salary_min'] = data['salary'].apply(deal, pattern=r"(\d+)k-(\d+)k", i=0)
data['salary_max'] = data['salary'].apply(deal, pattern=r"(\d+)k-(\d+)k", i=1)
data["salary_min"] = data["salary_min"].astype(int)
data["salary_max"] = data["salary_max"].apply(int)
data["salary_mid"] = (data["salary_min"] + data["salary_max"]) / 2

In [12]:
data["describe_cut"] = data["describe_cut"].str.lower()

In [13]:
data



,company_id,position_name,salary,describe_cut,salary_min,salary_max,salary_mid
0,53,中级/高级DBA,16k-30k,中级 高级 dba,16,30,23.0
1,53,PHP开发工程师,15k-25k,php 开发 工程师,15,25,20.0
2,53,IT工程师,5k-8k,it 工程师,5,8,6.5
3,53,IT技术支持,5k-7k,it 技术支持,5,7,6.0
4,53,python开发工程师,25k-30k,python 开发 工程师,25,30,27.5
...,...,...,...,...,...,...,...
130439,122184318,软件高级开发工程师,8k-15k,软件 高级 开发 工程师,8,15,11.5
130440,122184318,软件高级开发工程师,8k-15k,软件 高级 开发 工程师,8,15,11.5
130441,122184318,UI高级设计,11k-12k,ui 高级 设计,11,12,11.5
130442,122185826,vivo手机销售顾问,4k-8k,vivo 手机 销售 顾问,4,8,6.0


In [14]:
words_sum = dict()
for word in count_vec.get_feature_names():
    num = data.iloc[dict_word[word], 6].sum()/ len(dict_word[word])
    words_sum[word] = words_sum.get(word, (num if num is not None else 0) )

d:\code\python\statistics\venv\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
words_sum

{'ai': 26.11111111111111,
 'android': 22.754290171606865,
 'bd': 11.82312925170068,
 'hrbp': 16.553299492385786,
 'ios': 24.189688715953306,
 'java': 22.186233014034308,
 'mj': nan,
 'php': 17.800875273522976,
 'ui': 16.628348214285715,
 'web': 19.260187040748164,
 '上海': 25.085365853658537,
 '专员': 8.919126406353408,
 '专家': 34.88086398920014,
 '业务': 24.32072712076891,
 '中心': 26.945153061224488,
 '中级': 15.245833333333334,
 '主播': 11.350538599640933,
 '主管': 15.139137857447716,
 '产品': 25.777647439527545,
 '人事': 6.562642369020502,
 '代表': 9.054426705370101,
 '优化': 17.947100712105797,
 '内容': 18.624339894383102,
 '分析师': 22.640452538631347,
 '前端': 23.15727496026586,
 '前端开发': 22.12374749498998,
 '助理': 7.882259767687434,
 '北京': 26.662745098039217,
 '司机': 14.389830508474576,
 '后台': 29.6615873015873,
 '品牌': 21.886506935687265,
 '售前': 17.706436420722135,
 '商业': 28.785897435897436,
 '商业化': 29.327662721893493,
 '商务': 15.778428093645484,
 '培训': 14.367307692307692,
 '增长': 28.106047700170357,
 '媒体': 13.61

In [16]:
def deal(words: str, words_sum):
    words = words.split(" ")
    s = 0
    n = 0
    for word in words:

        if words_sum.get(word) is not None:
            s += words_sum.get(word)
            n += 1

    if n != 0:
        return s / n
    else:
        return s
(data["describe_cut"].apply(deal, words_sum=words_sum)==0).value_counts()

False    116924
True      13520
Name: describe_cut, dtype: int64

In [17]:
data["describe_cut"].apply(deal, words_sum=words_sum)

0         21.167774
1         20.783457
2          0.000000
3          0.000000
4         23.766038
            ...    
130439    23.016776
130440    23.016776
130441    21.859031
130442    12.567412
130443     0.000000
Name: describe_cut, Length: 130444, dtype: float64

In [18]:
116924 / (13520 + 116924)

0.8963539909846371

In [19]:
ls = []
for key in dict_word:
    ls += dict_word[key]
fitter = set(ls)

dict_other = {
    "other": []
}

for i in range(130443):
    if i not in fitter:
        dict_other["other"].append(i)

In [20]:
a = data.iloc[dict_other["other"], 6].sum() / len(dict_other["other"])

In [21]:
(data["describe_cut"].apply(deal, words_sum=words_sum).apply(lambda x: a if x == 0 or x is not None else x) == 0).value_counts()

False    130444
Name: describe_cut, dtype: int64

In [22]:
data["describe_cut_"] = data["describe_cut"].apply(deal, words_sum=words_sum).apply(lambda x: a if x == 0 or x is not None else x)

In [23]:
data["describe_cut"].apply(deal, words_sum=words_sum).apply(lambda x: a if x == 0 or x is not None else x).isnull().value_counts()

False    130444
Name: describe_cut, dtype: int64

In [24]:
data["describe_cut_"].to_csv("./职位关联.csv")

In [28]:
from sklearn.metrics import r2_score
-r2_score(data["salary_mid"], data["describe_cut_"])



0.2252922181308139